In [9]:
import os
from heapq import nlargest
from collections import Counter
from string import punctuation

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

import nltk

In [36]:
master_array = [] #append results to array to create data frame

for filename in os.listdir('season/'):
    if filename.endswith(".html"):
        split_name = filename.split('.') #obtain the season and episodes
        season = split_name[0]   
        episode = split_name[1]

        master_array.append([season, episode])
    else:
        pass
    
    
numblines = dict()
numblines["Episode"] = []
numblines["Number of lines"] = []
numblines["Character"] = []

worddicts = {}
worddicts['Monica'] = {}
worddicts["Rachel"]={}
worddicts["Phoebe"]={}
worddicts["Chandler"]={} 
worddicts["Ross"]={} 
worddicts["Joey"]={}

for num, ep in enumerate(master_array):
#for num in range(1):
    filename=master_array[num][0]+'.'+master_array[num][1]

    f = open('season/'+filename, 'r')
    data = f.read()

    from bs4 import BeautifulSoup
    soup = BeautifulSoup(data)
    readsoup = BeautifulSoup(data)

    import re

    soup = soup.get_text().translate(str.maketrans({'\n': ' ', '\xa0': ''}))
    pattern = re.compile(r'\s(?=\w+(?=:))') # store the regex
    result = re.split(pattern, soup) # split the script where our pattern matched (pink dot)


    episode_array=[]
    for item in result:
        split_line = item.split(': ')
        try:
            character = split_line[0]
            speech = split_line[1]
            episode_array.append([character, speech])
        except:
            pass

    import pandas as pd
    df = pd.DataFrame(episode_array, columns = ['Person','Said'])

    df['Person'].replace({'CHAN':'Chandler','CHANDLER':'Chandler', 'Chandlers':'Chandler',
                        'JOEY':'Joey',
                        'MNCA':'Monica','MONICA':'Monica',
                        'PHOE':'Phoebe','PHOEBE':'Phoebe', 'Pheebs':'Phoebe',
                        'Rache':'Rachel','RACHEL':'Rachel', 'RACH':'Rachel',
                        'ROSS':'Ross'},inplace=True)

    curved = re.compile("(\(.*?\))")
    brackets = re.compile("(\[.*?\])")
    scene = re.compile("\[Scene")
    for j,i in enumerate(df['Said']):
        df.iloc[j][1] =re.sub(curved, '', i)

    for j,i in enumerate(df['Said']):
        df.iloc[j][1] =re.sub(brackets, '', i)
        
    for j,i in enumerate(df['Said']):
        df.iloc[j][1] =re.sub(scene, '', i)



    namedict=dict()

    namedict["Rachel"]=0
    namedict["Phoebe"]=0
    namedict["Chandler"]=0 
    namedict["Ross"]=0 
    namedict["Joey"]=0
    namedict["Monica"]=0
    
    for i, j in enumerate(df["Person"]):
        if j in worddicts:
            wordlist = Counter((x.rstrip(punctuation).lower() for x in df.iloc[i][1].split()))
            for word in wordlist:
                try:
                    worddicts[j][word] = worddicts[j][word]+1
                except KeyError:
                    worddicts[j][word] = 1
    
    
    import matplotlib.pyplot as plt

    for i in df["Person"]:
        if i in namedict:
            namedict[i]=namedict[i]+1
    
    for p in namedict:
        try:
            numblines["Number of lines"].append(namedict[p])
            numblines["Character"].append(p)
            numblines["Episode"].append(master_array[num][0])
        except:
                pass
epinfo = pd.DataFrame.from_dict(numblines, orient='index')
epinfo = epinfo.transpose()
epinfo


,Episode,Number of lines,Character
0,0101,49,Rachel
1,0101,19,Phoebe
2,0101,39,Chandler
3,0101,47,Ross
4,0101,41,Joey
...,...,...,...
1369,1017-1018,85,Phoebe
1370,1017-1018,83,Chandler
1371,1017-1018,112,Ross
1372,1017-1018,84,Joey


In [46]:
wordlist = Counter((x.rstrip(punctuation).lower() for x in df.iloc[0][1].split()))
wordlist

Counter({'absolute': 1, 'margin': 1})

In [8]:
worddicts = {}
worddicts['Monica'] = {}
worddicts["Rachel"]={}
worddicts["Phoebe"]={}
worddicts["Chandler"]={} 
worddicts["Ross"]={} 
worddicts["Joey"]={}

for i, j in enumerate(df["Person"]):
    if j in worddicts:
        wordlist = Counter((x.rstrip(punctuation).lower() for x in df.iloc[i][1].split()))
        for word in wordlist:
            try:
                worddicts[j][word] = worddicts[j][word]+1
            except KeyError:
                worddicts[j][word] = 1
worddicts['Monica']


{'erica': 2,
 'are': 6,
 'you': 25,
 'okay': 7,
 'oh': 16,
 'my': 6,
 'god': 5,
 'she': 1,
 "doesn't": 2,
 'have': 4,
 'a': 9,
 'stomach-ache': 1,
 "she's": 1,
 'in': 5,
 'labor': 1,
 'breathe': 1,
 'good': 3,
 'no': 2,
 'last': 1,
 'time': 2,
 'said': 1,
 'it': 12,
 'like': 1,
 'dracula': 1,
 'and': 3,
 'scared': 1,
 'her': 1,
 'can': 2,
 'i': 13,
 'get': 3,
 'anything': 1,
 'want': 2,
 'some': 2,
 'more': 2,
 'ice': 1,
 'chips': 1,
 'alright': 5,
 "i'll": 4,
 'be': 8,
 'right': 1,
 'back': 3,
 'to': 8,
 'use': 1,
 'the': 14,
 'bathroom': 1,
 'what': 9,
 'kind': 1,
 'of': 5,
 'social': 1,
 'situation': 1,
 'comfortable': 1,
 'with': 4,
 "you'll": 1,
 'fine': 1,
 'nah': 1,
 "won't": 1,
 'but': 1,
 'two': 2,
 'minutes': 1,
 "it's": 6,
 'just': 5,
 'little': 3,
 'bit': 1,
 'honey': 2,
 'that': 10,
 'is': 11,
 'most': 1,
 'beautiful': 2,
 'top': 1,
 'head': 1,
 'ever': 2,
 'seen': 1,
 'chandler': 3,
 'see': 4,
 'this': 7,
 "don't": 2,
 'wanna': 2,
 'miss': 1,
 'birth': 1,
 'your': 2,
 'ch

In [47]:
personer = ['Monica','Rachel','Phoebe','Chandler','Ross','Joey']

sorterad2=dict(sorted(worddicts['Phoebe'].items(),reverse=True, key=lambda item: item[1]))
common_val_Phoebe = list(sorterad2.values())
common_keys_Phoebe = list(sorterad2.keys())

sorterad3=dict(sorted(worddicts['Rachel'].items(),reverse=True, key=lambda item: item[1]))
common_val_Rachel = list(sorterad3.values())
common_keys_Rachel = list(sorterad3.keys())

sorterad4=dict(sorted(worddicts['Chandler'].items(),reverse=True, key=lambda item: item[1]))
common_val_Chandler = list(sorterad4.values())
common_keys_Chandler = list(sorterad4.keys())

sorterad5=dict(sorted(worddicts['Ross'].items(),reverse=True, key=lambda item: item[1]))
common_val_Ross = list(sorterad5.values())
common_keys_Ross = list(sorterad5.keys())

sorterad6=dict(sorted(worddicts['Joey'].items(),reverse=True, key=lambda item: item[1]))
common_val_Joey = list(sorterad6.values())
common_keys_Joey = list(sorterad6.keys())



mostCommon = {}
numberOfWords = 10
for k in worddicts.keys():
    l = sorted(worddicts[k].items(), key=lambda item: item[1], reverse=True)[0:numberOfWords]
    for word in l:
        if word[0] in mostCommon:
            mostCommon.update({word[0] : mostCommon[word[0]]+word[1]})
        else:
            mostCommon.update({word[0] : word[1]})

            
sorterad1=dict(sorted(worddicts['Monica'].items(),reverse=True, key=lambda item: item[1]))
common_val_Monica = list(sorterad1.values())
common_keys_Monica = list(sorterad1.keys())            

MonicaKorr = [0]*len(list(mostCommon.keys()))
RachelKorr = [0]*len(list(mostCommon.keys()))
PhoebeKorr = [0]*len(list(mostCommon.keys()))
JoeyKorr = [0]*len(list(mostCommon.keys()))
ChandlerKorr = [0]*len(list(mostCommon.keys()))
RossKorr = [0]*len(list(mostCommon.keys()))


most_common_all_keys = list(mostCommon.keys())

print('Vanligaste ord alla')
print(most_common_all_keys)
print('Antal ggr sagda')
print(list(mostCommon.values()))

#Monica
print('Monicas 25 vanligaste ord')
print(common_keys_Joey[:numberOfWords])
print('Monicas 25 vanligaste ord values')
print(common_val_Joey[:numberOfWords])

for i in range(len(most_common_all_keys)):
    for j in common_keys_Joey[:numberOfWords]:
        if most_common_all_keys[i]==j:
            MonicaKorr[i]=common_val_Ross[common_keys_Ross.index(j)]
        else:
            continue

monica_delat_med_all = []
for k in range(len(most_common_all_keys)):  
    monica_delat_med_all.append(round(MonicaKorr[k]/list(mostCommon.values())[k],2))
            
print('Monicakorr:')
print(MonicaKorr)

print('Procent av mest sagda orden monica utgör:')
print(monica_delat_med_all)
    
#Phoebe
print('Phoebe 25 vanligaste ord')
print(common_keys_Phoebe[:numberOfWords])
print('Phoebe 25 vanligaste ord values')
print(common_val_Phoebe[:numberOfWords])

for i in range(len(most_common_all_keys)):
    for j in common_keys_Phoebe[:numberOfWords]:
        if most_common_all_keys[i]==j:
            PhoebeKorr[i]=common_val_Phoebe[common_keys_Phoebe.index(j)]
        else:
            continue

Phoebe_delat_med_all = []
for k in range(len(most_common_all_keys)):  
    Phoebe_delat_med_all.append(round(PhoebeKorr[k]/list(mostCommon.values())[k],2))
            
print('Phoebekorr:')
print(PhoebeKorr)

print('Procent av mest sagda orden Phoebe utgör:')
print(Phoebe_delat_med_all)

Vanligaste ord alla
['you', 'i', 'the', 'to', 'a', 'it', 'and', 'that', 'what', 'oh', 'so', 'is', 'hey', 'yeah']
Antal ggr sagda
[15263, 13759, 9363, 8473, 8109, 6061, 6459, 6093, 3890, 3764, 784, 1641, 927, 842]
Monicas 25 vanligaste ord
['you', 'i', 'the', 'a', 'to', 'and', 'it', 'that', 'hey', 'yeah']
Monicas 25 vanligaste ord values
[2405, 2340, 1649, 1389, 1361, 1091, 1077, 956, 927, 842]
Monicakorr:
[2574, 2469, 1630, 1530, 1468, 1069, 1077, 938, 0, 0, 0, 0, 669, 756]
Procent av mest sagda orden monica utgör:
[0.17, 0.18, 0.17, 0.18, 0.18, 0.18, 0.17, 0.15, 0.0, 0.0, 0.0, 0.0, 0.72, 0.9]
Phoebe 25 vanligaste ord
['you', 'i', 'the', 'oh', 'to', 'a', 'and', 'that', 'it', 'so']
Phoebe 25 vanligaste ord values
[2362, 1959, 1394, 1273, 1235, 1232, 1065, 922, 864, 784]
Phoebekorr:
[2362, 1959, 1394, 1235, 1232, 864, 1065, 922, 0, 1273, 784, 0, 0, 0]
Procent av mest sagda orden Phoebe utgör:
[0.15, 0.14, 0.15, 0.15, 0.15, 0.14, 0.16, 0.15, 0.0, 0.34, 1.0, 0.0, 0.0, 0.0]
